In [ ]:
!pip install selenium

In [ ]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
import imp
from lib2to3.pgen2 import driver
from sqlite3 import Time
from time import sleep
from selenium import webdriver
from bs4 import BeautifulSoup
import json
import pandas as pd
from tqdm import tqdm

import requests

In [ ]:
scrollnum = 100
sleepTimer = 3
chromeDriverPath = r"C:\Users\moham\Image-Based_Semantic_Search\Images_scraping\chromedriver.exe"

# choose the field of interest in scraping 
var = ['drink', 'soda', 'juice', 'drinkable', 'icecream', 'mokhito', 'smoothie']



In [ ]:
profiles = []
image_links = {}
for variable in var:
    print(variable)
    url = f'https://www.pinterest.fr/search/pins/?q={variable}'
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(executable_path=chromeDriverPath,options=options)
    driver.get(url)
    
    for _  in tqdm(range(1, scrollnum)):
        driver.execute_script("window.scrollTo(1,100000)")
        print("scroll down")
        soup = BeautifulSoup(driver.page_source, 'html.parser')


        # find profiles pictures to put them out of the dataset
        listToStr = ' '.join([str(elem) for elem in soup.find_all("div", {"class": "Pj7 sLG XiG INd m1e"})])

        profile_pics = BeautifulSoup(listToStr, 'html.parser')
        for prof in profile_pics.findAll('img'):
            profiles.append(prof['src'])
            # print(prof['src'])


        # extract all images from the page
        
        for link in soup.findAll('img'):
            
            try:
              image_links[link['src']] = link['alt']
            except:
              pass
        
        sleep(sleepTimer)
    


In [ ]:
with open('data.json', 'w') as fp:
    json.dump(image_links, fp)

In [ ]:
for item in profiles:
    try:
        del image_links[item]
    except:
        pass


In [ ]:
with open('data.json') as json_file:
    data = json.load(json_file)

In [ ]:
df = pd.DataFrame(list(data.items()), columns = ['image_link','image_description'])
df.to_csv('scraped_data.csv')

In [ ]:
!mkdir images

In [ ]:
df.head()

,image_link,image_description
0,https://i.pinimg.com/236x/d6/fa/05/d6fa05a7285...,Golden sparkling water ✨
1,https://i.pinimg.com/236x/53/7f/7a/537f7a51a39...,Pop Fizz Clink | Natalie Grace Tatum | aesthet...
2,https://i.pinimg.com/236x/31/e6/e6/31e6e62befb...,The Best Virgin Strawberry Daiquiri You'll Eve...
3,https://i.pinimg.com/236x/15/5d/93/155d934d670...,Easy Piña Colada
4,https://i.pinimg.com/236x/e9/30/32/e93032193ac...,3-Ingredient Korean Strawberry Milk


In [ ]:
urls = list(data.keys())
for i in tqdm(range(len(urls)), desc = 'Downloading images'):
  response = requests.get(urls[i])
  filename = urls[i].split('/')[-1]

  if response.status_code == 200:
    with open('/content/images/' + filename, "wb") as image:
      for chunk in response.iter_content(chunk_size=128):
        image.write(chunk)

In [ ]:
dataset = pd.DataFrame([(l.split('/')[-1], k) for l, k in data.items()], columns = ['id','description'])
dataset['index'] = range(1, len(dataset) + 1)
dataset.to_csv('dataset.csv')

In [ ]:
!zip -r data.zip images dataset.csv